In [1]:
import os
import torch
import numpy as np
import pandas as pd
import sys
import random
import h5py
import torch
import torchvision
import ast
from torch.utils.tensorboard import SummaryWriter

sys.path.append('../')

from models.mcnnpytorch.src import *
from models.mcnnpytorch.src.crowd_count import *
from models.mcnnpytorch.src.network import *
from models.mcnnpytorch.src.data_loader import ImageDataLoader
from models.mcnnpytorch.src.timer import *
from models.mcnnpytorch.src.evaluate_model import *
from models.mcnnpytorch.src import utils

In [2]:
# Check to see if device can be trained on GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(device)

cuda


In [3]:
# Cuda configurations

torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

current_device = torch.cuda.current_device()
current_device_name = torch.cuda.get_device_name(current_device)
memory_allocated = torch.cuda.memory_allocated()
memory_cached = torch.cuda.memory_cached()

print(
    f'Using gpu {current_device_name} with device number {current_device}.\n'
    f'Memory allocated = {memory_allocated}\n'
    f'Memory cached = {memory_cached}'
)

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [4]:
try:
    from termcolor import cprint
except ImportError:
    cprint = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)

In [5]:
# Directory Configurations

method = 'mcnn'
dataset_name = 'JHU'
output_dir = f'../output/{method}/saved_models/{dataset_name}'

# Training data path
train_path = '../data/JHU/train/consolidated'
train_gt_path = '../data/JHU/train/gt'

# Validation data path
val_path = '../data/JHU/val/consolidated'
val_gt_path = '../data/JHU/val/gt'

In [6]:
# Create output directory if it doesnt exist

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [7]:
# load model

is_cuda = True  # Determine if we should use the CPU to train or GPU

model = CrowdCounter(is_cuda=is_cuda)  # is_cuda determines if all the input tensors should be converted to cuda tensors
network.weights_normal_init(model, dev=0.01)
model.train()

CrowdCounter(
  (model): MCNN(
    (branch1): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Conv2d(
        (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(
        (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (5): Conv2d(
        (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
    )
    (branch2): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d

In [8]:
# Model parameters

for name, param in model.named_parameters():
    print(f'{name}\t{param.device}\t{param.shape}')

# Print model's state_dict
print("\nModel's state_dict: \n")
for k, v in model.state_dict().items():
    print(k, "\t", v.dtype)

model.branch1.0.conv.weight	cpu	torch.Size([16, 3, 9, 9])
model.branch1.0.conv.bias	cpu	torch.Size([16])
model.branch1.2.conv.weight	cpu	torch.Size([32, 16, 7, 7])
model.branch1.2.conv.bias	cpu	torch.Size([32])
model.branch1.4.conv.weight	cpu	torch.Size([16, 32, 7, 7])
model.branch1.4.conv.bias	cpu	torch.Size([16])
model.branch1.5.conv.weight	cpu	torch.Size([8, 16, 7, 7])
model.branch1.5.conv.bias	cpu	torch.Size([8])
model.branch2.0.conv.weight	cpu	torch.Size([20, 3, 7, 7])
model.branch2.0.conv.bias	cpu	torch.Size([20])
model.branch2.2.conv.weight	cpu	torch.Size([40, 20, 5, 5])
model.branch2.2.conv.bias	cpu	torch.Size([40])
model.branch2.4.conv.weight	cpu	torch.Size([20, 40, 5, 5])
model.branch2.4.conv.bias	cpu	torch.Size([20])
model.branch2.5.conv.weight	cpu	torch.Size([10, 20, 5, 5])
model.branch2.5.conv.bias	cpu	torch.Size([10])
model.branch3.0.conv.weight	cpu	torch.Size([24, 3, 5, 5])
model.branch3.0.conv.bias	cpu	torch.Size([24])
model.branch3.2.conv.weight	cpu	torch.Size([48, 24,

In [9]:
# Change model weights tensors to be cuda tensors if is_cuda is true and cuda is available

if is_cuda and torch.cuda.is_available():
    print("Changing to cuda weights")
    model.cuda()

Changing to cuda weights


In [10]:
#training configuration

#momentum = 0.9
disp_interval = 10
#log_interval = 250

train_loss = 0
re_cnt = False
t = Timer()
t.tic()

# Set initial values
best_mae, best_mse, best_epoch = 999999, 999999, 0

In [11]:
# Hyperparameters

learning_rate = 0.0001
epochs = 100

# construct an optimizer

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=learning_rate)

In [12]:
# Load the images, take note the num_pool argument

data_loader = ImageDataLoader(train_path, shuffle=False, pre_load=False, num_pool = 2 )
data_loader_val = ImageDataLoader(val_path, shuffle=False, pre_load=False, num_pool = 2)

print('Training instances: {}'.format(data_loader.get_num_samples()))
print('Validation instances: {}'.format(data_loader_val.get_num_samples()))

Training instances: 500
Validation instances: 100


In [17]:
#Tensorboard  config
use_tensorboard = True

writer = SummaryWriter(f'../output/tensorboard/runs/{learning_rate}_{epochs}')

layout = {
    
    'MAE': {'Weather': ['Margin', ['Weather/None', 'Weather/Fog', 'Weather/Rain', 'Weather/Snow']],
           'Crowd Density': ['Margin', ['Crowd Density/Low', 'Crowd Density/Med', 'Crowd Density/High']]},
    
    'MSE': {'Weather': ['Margin', ['Weather/None', 'Weather/Fog', 'Weather/Rain', 'Weather/Snow']],
           'Crowd Density': ['Margin', ['Crowd Density/Low', 'Crowd Density/Med', 'Crowd Density/High']]}
}

#writer.add_custom_scalars(layout)

In [14]:
for epoch in range(epochs):    
    step = -1
    train_loss = 0
    for id, blob in enumerate(data_loader):                
        step = step + 1        
        im_data = blob['data']
        gt_data = blob['gt_density']
        metadata = blob['metadata']
                        
        # Forward pass
        density_map = model(im_data, gt_data)
        
        loss = model.loss
        train_loss += loss.data
        
        # Write to tensorboard
        writer.add_scalar("Loss/train", train_loss, epoch)

        # Reset zero gradient and backpropagate
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % disp_interval == 0:            
            duration = t.toc(average=False)
            
            gt_count = np.sum(gt_data)    

            density_map = density_map.data.cpu().numpy()
            et_count = np.sum(density_map)
            utils.save_results(im_data,gt_data,density_map, output_dir)
            log_text = 'epoch: %4d, step %4d, Time: %.4fs, gt_cnt: %7.1f, et_cnt: %7.1f' % (epoch,step,duration, gt_count, et_count)
            log_print(log_text, color='green', attrs=['bold'])
            re_cnt = True
       
        if re_cnt:                                
            t.tic()
            re_cnt = False
        

    # Overwrite the current model weights
    current_model = f'{method}_{learning_rate}.h5'
    save_name = os.path.join(output_dir, current_model)
    network.save_net(save_name, model)
            
    # Evaluate the mae and mse results by doing a forward pass against the validation dataset i.e data_loader_val for
    # each epoch
    MAEcrowddensity, MSEcrowddensity, MAEweather, MSEweather, MAE, MSE = evaluate_model(save_name, data_loader_val, is_cuda=is_cuda)
    
    # Pocket algorithm: Check to see if the current epoch mae is better than the best recorded one,
    # If it is, then overwrite the current best .h5 weights file
    if MAE < best_mae:
        # Save the new best mae and mse
        best_mae = MAE
        best_mse = MSE
        best_epoch = epoch
        best_model = f'best_model_epoch_{best_epoch}_{method}_{learning_rate}.h5'

        # Overwrite or create a new file for the best model for this learning rate
        save_name = os.path.join(output_dir, best_model)
        network.save_net(save_name, model)
        
    # Print out the best epoch that beat the current best mae
    log_text = f'EPOCH: {epoch}, MAE: {MAE}, MSE: {MSE}'
    log_print(log_text, color='blue', attrs=['bold'])

    # Save the results to tensorboard for each epoch
    if use_tensorboard:
        
        # overall segment
        writer.add_scalar("Overall/MAE", MAE, epoch)
        writer.add_scalar("Overall/MSE", train_loss, epoch)
        writer.add_scalar("Overall/train_loss", train_loss / data_loader.get_num_samples(), epoch)
        
        # crowd density segment
        writer.add_scalar('High/MAE', MAEcrowddensity['High'], epoch)
        writer.add_scalar('High/MSE', MSEcrowddensity['High'], epoch)
        
        writer.add_scalar('Med/MAE', MAEcrowddensity['Med'], epoch)
        writer.add_scalar('Med/MSE', MSEcrowddensity['Med'], epoch)
        
        writer.add_scalar('Low/MAE', MAEcrowddensity['Low'], epoch)
        writer.add_scalar('Low/MSE', MSEcrowddensity['Low'], epoch)
        
        # weather segment
        writer.add_scalar('None/MAE', MAEweather['None'], epoch)
        writer.add_scalar('None/MSE', MSEweather['None'], epoch)
        
        writer.add_scalar('Fog/MAE', MAEweather['Fog'], epoch)
        writer.add_scalar('Fog/MSE', MSEweather['Fog'], epoch)
        
        writer.add_scalar('Rain/MAE', MAEweather['Rain'], epoch)
        writer.add_scalar('Rain/MSE', MSEweather['Rain'], epoch)
        
        writer.add_scalar('Snow/MAE', MAEweather['Snow'], epoch)
        writer.add_scalar('Snow/MSE', MSEweather['Snow'], epoch)

epoch:    0, step    0, Time: 1.4661s, gt_cnt:    67.0, et_cnt:     0.0
epoch:    0, step   10, Time: 2.3245s, gt_cnt:   270.9, et_cnt:    67.2
epoch:    0, step   20, Time: 2.4216s, gt_cnt:    18.4, et_cnt:   119.5
epoch:    0, step   30, Time: 2.4610s, gt_cnt:    26.1, et_cnt:   224.4
epoch:    0, step   40, Time: 2.2959s, gt_cnt:    71.7, et_cnt:   464.0
epoch:    0, step   50, Time: 2.5466s, gt_cnt:   424.4, et_cnt:   208.5
epoch:    0, step   60, Time: 2.3088s, gt_cnt:   447.1, et_cnt:   128.0
epoch:    0, step   70, Time: 2.2972s, gt_cnt:   579.7, et_cnt:   216.4
epoch:    0, step   80, Time: 2.7432s, gt_cnt:     1.0, et_cnt:   239.0
epoch:    0, step   90, Time: 2.3030s, gt_cnt:   141.8, et_cnt:   320.2
epoch:    0, step  100, Time: 2.5377s, gt_cnt:   131.4, et_cnt:   199.0
epoch:    0, step  110, Time: 2.3082s, gt_cnt:    21.7, et_cnt:   272.2
epoch:    0, step  120, Time: 2.2971s, gt_cnt:   472.4, et_cnt:   358.2
epoch:    0, step  130, Time: 2.5951s, gt_cnt:   163.6, et_cnt: 

../models/mcnnpytorch/src/network.py:64: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  v = Variable(torch.as_tensor(x).type(dtype), requires_grad = False, volatile = True)


EPOCH: 0, MAE: 303.81285552978517, MSE: 219870.06469384194
epoch:    1, step    0, Time: 13.8416s, gt_cnt:    67.0, et_cnt:   304.4
epoch:    1, step   10, Time: 2.3214s, gt_cnt:   270.9, et_cnt:   305.7
epoch:    1, step   20, Time: 2.4351s, gt_cnt:    18.4, et_cnt:   311.9
epoch:    1, step   30, Time: 2.4407s, gt_cnt:    26.1, et_cnt:   308.5
epoch:    1, step   40, Time: 2.3047s, gt_cnt:    71.7, et_cnt:   313.5
epoch:    1, step   50, Time: 2.4986s, gt_cnt:   424.4, et_cnt:   308.1
epoch:    1, step   60, Time: 2.3230s, gt_cnt:   447.1, et_cnt:   298.6
epoch:    1, step   70, Time: 2.3235s, gt_cnt:   579.7, et_cnt:   307.3
epoch:    1, step   80, Time: 2.7297s, gt_cnt:     1.0, et_cnt:   314.0
epoch:    1, step   90, Time: 2.3098s, gt_cnt:   141.8, et_cnt:   313.0
epoch:    1, step  100, Time: 2.5242s, gt_cnt:   131.4, et_cnt:   309.0
epoch:    1, step  110, Time: 2.3027s, gt_cnt:    21.7, et_cnt:   314.7
epoch:    1, step  120, Time: 2.3200s, gt_cnt:   472.4, et_cnt:   318.2
epoc

epoch:    2, step  450, Time: 2.3122s, gt_cnt:    31.2, et_cnt:   367.5
epoch:    2, step  460, Time: 2.5010s, gt_cnt:    67.1, et_cnt:   353.7
epoch:    2, step  470, Time: 2.4038s, gt_cnt:   561.6, et_cnt:   334.2
epoch:    2, step  480, Time: 2.3117s, gt_cnt:   332.2, et_cnt:   331.2
epoch:    2, step  490, Time: 2.3119s, gt_cnt:   369.8, et_cnt:   340.3
EPOCH: 2, MAE: 321.50773864746094, MSE: 221939.35202694684
epoch:    3, step    0, Time: 13.3463s, gt_cnt:    67.0, et_cnt:   338.3
epoch:    3, step   10, Time: 2.2946s, gt_cnt:   270.9, et_cnt:   336.3
epoch:    3, step   20, Time: 2.4058s, gt_cnt:    18.4, et_cnt:   339.7
epoch:    3, step   30, Time: 2.4303s, gt_cnt:    26.1, et_cnt:   333.3
epoch:    3, step   40, Time: 2.2946s, gt_cnt:    71.7, et_cnt:   336.4
epoch:    3, step   50, Time: 2.5109s, gt_cnt:   424.4, et_cnt:   327.8
epoch:    3, step   60, Time: 2.3216s, gt_cnt:   447.1, et_cnt:   316.1
epoch:    3, step   70, Time: 2.3103s, gt_cnt:   579.7, et_cnt:   323.3
epoc

epoch:    4, step  400, Time: 2.6137s, gt_cnt:     0.4, et_cnt:   454.4
epoch:    4, step  410, Time: 2.3002s, gt_cnt:    79.9, et_cnt:   434.2
epoch:    4, step  420, Time: 2.2948s, gt_cnt:   311.5, et_cnt:   417.4
epoch:    4, step  430, Time: 2.3070s, gt_cnt:   202.1, et_cnt:   403.0
epoch:    4, step  440, Time: 2.3400s, gt_cnt:   275.0, et_cnt:   384.5
epoch:    4, step  450, Time: 2.2974s, gt_cnt:    31.2, et_cnt:   369.4
epoch:    4, step  460, Time: 2.4932s, gt_cnt:    67.1, et_cnt:   358.3
epoch:    4, step  470, Time: 2.4014s, gt_cnt:   561.6, et_cnt:   346.0
epoch:    4, step  480, Time: 2.2912s, gt_cnt:   332.2, et_cnt:   342.6
epoch:    4, step  490, Time: 2.3015s, gt_cnt:   369.8, et_cnt:   348.7
EPOCH: 4, MAE: 326.0750401306152, MSE: 222763.64897750854
epoch:    5, step    0, Time: 13.3014s, gt_cnt:    67.0, et_cnt:   346.2
epoch:    5, step   10, Time: 2.2935s, gt_cnt:   270.9, et_cnt:   343.8
epoch:    5, step   20, Time: 2.4096s, gt_cnt:    18.4, et_cnt:   346.1
epoch

...

epoch:   94, step   90, Time: 2.2570s, gt_cnt:   141.8, et_cnt:   142.7
epoch:   94, step  100, Time: 2.4674s, gt_cnt:   131.4, et_cnt:   523.5
epoch:   94, step  110, Time: 2.2582s, gt_cnt:    21.7, et_cnt:     0.8
epoch:   94, step  120, Time: 2.2575s, gt_cnt:   472.4, et_cnt:   403.0
epoch:   94, step  130, Time: 2.5014s, gt_cnt:   163.6, et_cnt:   394.8
epoch:   94, step  140, Time: 2.2566s, gt_cnt:  2686.5, et_cnt:   419.1
epoch:   94, step  150, Time: 2.3711s, gt_cnt:  2206.5, et_cnt:   644.4
epoch:   94, step  160, Time: 2.4982s, gt_cnt:    73.0, et_cnt:   374.5
epoch:   94, step  170, Time: 2.2982s, gt_cnt:   583.4, et_cnt:   454.0
epoch:   94, step  180, Time: 2.3434s, gt_cnt:   337.5, et_cnt:   264.9
epoch:   94, step  190, Time: 2.2611s, gt_cnt:   401.4, et_cnt:   204.0
epoch:   94, step  200, Time: 2.2550s, gt_cnt:   167.1, et_cnt:   479.7
epoch:   94, step  210, Time: 2.2622s, gt_cnt:  1480.8, et_cnt:   443.7
epoch:   94, step  220, Time: 2.5571s, gt_cnt:    86.7, et_cnt: 

../models/mcnnpytorch/src/utils.py:8: RuntimeWarning: invalid value encountered in true_divide
  density_map = 255 * density_map / np.max(density_map)


epoch:   95, step  110, Time: 2.2486s, gt_cnt:    21.7, et_cnt:     0.0
epoch:   95, step  120, Time: 2.2512s, gt_cnt:   472.4, et_cnt:     0.0
epoch:   95, step  130, Time: 2.5018s, gt_cnt:   163.6, et_cnt:     0.0
epoch:   95, step  140, Time: 2.2455s, gt_cnt:  2686.5, et_cnt:     0.0
epoch:   95, step  150, Time: 2.3636s, gt_cnt:  2206.5, et_cnt:     0.0
epoch:   95, step  160, Time: 2.4894s, gt_cnt:    73.0, et_cnt:     0.0
epoch:   95, step  170, Time: 2.2846s, gt_cnt:   583.4, et_cnt:     0.6
epoch:   95, step  180, Time: 2.3340s, gt_cnt:   337.5, et_cnt:     0.0
epoch:   95, step  190, Time: 2.2498s, gt_cnt:   401.4, et_cnt:     0.0
epoch:   95, step  200, Time: 2.2421s, gt_cnt:   167.1, et_cnt:     0.0
epoch:   95, step  210, Time: 2.2505s, gt_cnt:  1480.8, et_cnt:     0.0
epoch:   95, step  220, Time: 2.5599s, gt_cnt:    86.7, et_cnt:     0.0
epoch:   95, step  230, Time: 2.2627s, gt_cnt:    26.1, et_cnt:     0.4
epoch:   95, step  240, Time: 2.5365s, gt_cnt:   140.8, et_cnt: 

epoch:   97, step   60, Time: 2.2693s, gt_cnt:   447.1, et_cnt:   415.4
epoch:   97, step   70, Time: 2.2625s, gt_cnt:   579.7, et_cnt:   295.1
epoch:   97, step   80, Time: 2.6955s, gt_cnt:     1.0, et_cnt:     0.2
epoch:   97, step   90, Time: 2.2585s, gt_cnt:   141.8, et_cnt:   139.7
epoch:   97, step  100, Time: 2.4807s, gt_cnt:   131.4, et_cnt:   524.1
epoch:   97, step  110, Time: 2.2574s, gt_cnt:    21.7, et_cnt:     0.5
epoch:   97, step  120, Time: 2.2611s, gt_cnt:   472.4, et_cnt:   393.0
epoch:   97, step  130, Time: 2.5348s, gt_cnt:   163.6, et_cnt:   393.8
epoch:   97, step  140, Time: 2.2610s, gt_cnt:  2686.5, et_cnt:   390.0
epoch:   97, step  150, Time: 2.3759s, gt_cnt:  2206.5, et_cnt:   661.4
epoch:   97, step  160, Time: 2.5136s, gt_cnt:    73.0, et_cnt:   397.3
epoch:   97, step  170, Time: 2.3064s, gt_cnt:   583.4, et_cnt:   438.0
epoch:   97, step  180, Time: 2.3421s, gt_cnt:   337.5, et_cnt:   273.5
epoch:   97, step  190, Time: 2.2569s, gt_cnt:   401.4, et_cnt: 

epoch:   99, step   10, Time: 2.2680s, gt_cnt:   270.9, et_cnt:   243.4
epoch:   99, step   20, Time: 2.3774s, gt_cnt:    18.4, et_cnt:   182.4
epoch:   99, step   30, Time: 2.3854s, gt_cnt:    26.1, et_cnt:   432.3
epoch:   99, step   40, Time: 2.2568s, gt_cnt:    71.7, et_cnt:   307.2
epoch:   99, step   50, Time: 2.4670s, gt_cnt:   424.4, et_cnt:   470.1
epoch:   99, step   60, Time: 2.2654s, gt_cnt:   447.1, et_cnt:   431.9
epoch:   99, step   70, Time: 2.2623s, gt_cnt:   579.7, et_cnt:   275.8
epoch:   99, step   80, Time: 2.6931s, gt_cnt:     1.0, et_cnt:     0.1
epoch:   99, step   90, Time: 2.2580s, gt_cnt:   141.8, et_cnt:   136.5
epoch:   99, step  100, Time: 2.4821s, gt_cnt:   131.4, et_cnt:   544.0
epoch:   99, step  110, Time: 2.2529s, gt_cnt:    21.7, et_cnt:     0.7
epoch:   99, step  120, Time: 2.2599s, gt_cnt:   472.4, et_cnt:   404.6
epoch:   99, step  130, Time: 2.5047s, gt_cnt:   163.6, et_cnt:   391.7
epoch:   99, step  140, Time: 2.2591s, gt_cnt:  2686.5, et_cnt: 

In [16]:
log_text = f'BEST MAE: {best_mae}, BEST MSE: {best_mse}'
log_print(log_text, color='green', attrs=['bold'])

BEST MAE: 290.4987934780121, BEST MSE: 304073.0321883392
